In [92]:
import base64
import pandas as pd
import requests
import json
import spotipy.util as util
import spotipy
import os
import statistics as st

In [18]:
# import secrets for toke authenification 
import import_ipynb
from spotify_secret import *

In [19]:
# secrets
library = my_library_path
streams = my_streams_path

In [20]:
# required information for a authentication token 
scopes = ['user-read-recently-played', 'playlist-modify-private']

In [157]:
# get access toke (tokens only last 1 hour)
token = util.prompt_for_user_token(username=username, scope=scopes, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth = token)

In [158]:
#header for all api interactions
headers = {'Authorization': 'Bearer ' + token,'Content-Type': 'application/json',}
# boilerplate json for playlist creation
pl_name_data = {'name': '', 'description': 'New playlist description','public': False,}

In [23]:
# URLS
#recent
spotify_url = 'https://api.spotify.com/v1/me/player/recently-played'
#playlist
playlist_url = f'https://api.spotify.com/v1/users/{username}/playlists'

In [41]:
#paths for importing streamed songs files
stream_paths = [os.path.join(streams, item) for item in os.listdir(streams)]

In [42]:
stream_paths

['C:\\Users\\willd\\OneDrive\\Desktop\\DataAnalysis\\python data\\climate trends\\spotify data\\myStrams\\StreamingHistory0.json',
 'C:\\Users\\willd\\OneDrive\\Desktop\\DataAnalysis\\python data\\climate trends\\spotify data\\myStrams\\StreamingHistory1.json']

### Recently played

In [6]:
# recent tracks
request = requests.get(url = spotify_url, headers=headers)
recent = request.json()

### My Data 

In [43]:
# my library
with open(library+r'\YourLibrary.json') as json_data:
    data = json.load(json_data)
my_library = pd.DataFrame(data['tracks'])
my_library['art_track'] = my_library['artist'] + my_library['track']

In [145]:
# my listener history as a df 
list_of_df = []
for file in stream_paths:
    with open(file, encoding="utf8") as stream_json:
        stream_data = json.load(stream_json)
        my_streams = pd.DataFrame(stream_data)
        list_of_df.append(my_streams)
full_streams = pd.concat(list_of_df)

In [146]:
my_full_streams = full_streams
full_streams['endTime'] = pd.to_datetime(full_streams['endTime'])
my_full_streams['msPlayed'] = (my_full_streams['msPlayed'] / 1000)
my_full_streams = my_full_streams.rename(columns={'msPlayed' : 'Secs_played'})
#remove quickly skipped songs
my_full_streams = my_full_streams[my_full_streams['Secs_played'] > 8]
my_full_streams['art_track'] = my_full_streams['artistName'] + my_full_streams['trackName']
my_full_streams['endTime'] = pd.DatetimeIndex(my_full_streams['endTime']).year
my_full_streams['count'] = my_full_streams.groupby('art_track')['art_track'].transform('count')
# the year the track was played the most
my_full_streams['year'] = my_full_streams.groupby('art_track')['endTime'].transform(lambda x: st.mode(x))

In [130]:
# all years from data set for playlist specification
years = set(list(my_full_streams['year']))
years

{2022, 2023}

In [147]:
no_skips = no_skips[['trackName','artistName', 'art_track', 'year', 'count']]
no_skips = no_skips.drop_duplicates()
over_five = no_skips[no_skips['count'] >= 5]

In [148]:
over_five.shape

(901, 5)

In [149]:
not_liked = over_five[~over_five['art_track'].isin(my_library['art_track'])]

In [150]:
not_liked.shape

(201, 5)

In [151]:
not_liked = not_liked.reset_index(drop=True)

In [152]:
not_liked['uri'] = ''

In [159]:
for i in range(0, len(not_liked)):
    not_liked.loc[i,'uri'] = get_uri(not_liked.loc[i,'trackName'], not_liked.loc[i,'artistName'])

In [160]:
liked_not_liked = list(not_liked['uri'])

In [164]:
len(liked_not_liked)

201

In [169]:
liked_not_liked= list(set(liked_not_liked))

In [170]:
list1 = liked_not_liked[0:80]
list2 = liked_not_liked[80:160]
list3 = liked_not_liked[160:]

### Feature analysis

In [33]:
# get track features
features = sp.audio_features(uris_list)

In [32]:
# make features df
feature_df = pd.DataFrame.from_records(features)
#join to reduced df
full_df = reduced_df.join(feature_df.set_index('uri'), on='uri')

### Get required track information

In [156]:
def get_uri(track, artist):
    qq = f""
    search_song = sp.search(q=f'track: {track} artist: {artist}', type='track')
    track_uri=search_song['tracks']['items'][0]['uri']
    return track_uri

In [205]:
search_song=sp.search(q='track: Bad Dreams artist: Lyrics Born', type='track')

In [196]:
# for option in search_song['tracks']['items']:
#     if option['artists'][0]['name'] == 'Lyrics Born':
#         track_uri = option['uri']
        

### Playlist Creation & Modification 

In [154]:
# function to create playlist and return playlist id for modification
def new_playlist(playlist_name):
    global pl_id
    pl_name_data['name'] = playlist_name
    playlist_created = requests.post(playlist_url, headers=headers, json=pl_name_data)
    pl_obj = playlist_created.json()
    pl_id = pl_obj['id']

In [155]:
# function to update created plalists (only accepts lists of uris < 100)
def mod_playlist(pl_id, pl_uris):
    sp.playlist_add_items(pl_id, pl_uris, position=None)

In [171]:
# call function to create playlist
new_playlist('test 2')

In [172]:
# call function to add to newly created playlist 
mod_playlist(pl_id, list1)
mod_playlist(pl_id, list2)
mod_playlist(pl_id, list3)